In [1]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Подключаем к наше программе модуль с предыдущего шага. Этот модуль будет повторяться всегда. 
# Мы импортируем модуль psycopg2 и называем его pg_driver для удобства
import psycopg2 as pg_driver


# На данном шаге нам нужно попробовать подключить к нашей базе данных. При установке PostgreSQL вам предлагалось задать
# пароль от базы данных по умолчанию, собственно ниже вам следует изменить только password='postgres', но не удаляйте кавычки, это важно
db = pg_driver.connect(
    database="postgres", 
    user='postgres',
    password='546038', 
    host='localhost', 
    port='5432'
)

# Далее нам нужен курсор, который позволит нам непосредственно отправлять запросы на выполнения в БД. Эта строка всегда одинаковая
cur = db.cursor()
# Делаем запрос к таблице по умолчанию pg_database
cur.execute("select * from pg_database")

# Чтобы извлечь результат выполнения запроса к таблице по умолчанию, необходимо воспользоваться методом .fetchall()
rs = cur.fetchall()


# enumerate позволяет красиво пронумеровать строки
for row, values in enumerate(rs):
    print(row, values)

0 (13754, 'postgres', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', False, True, -1, 13753, '727', '1', 1663, None)
1 (16394, 'social_media', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', False, True, -1, 13753, '727', '1', 1663, None)
2 (1, 'template1', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', True, True, -1, 13753, '727', '1', 1663, '{=c/postgres,postgres=CTc/postgres}')
3 (13753, 'template0', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', True, False, -1, 13753, '727', '1', 1663, '{=c/postgres,postgres=CTc/postgres}')
4 (16395, 'testdb', 10, 6, 'Russian_Russia.1251', 'Russian_Russia.1251', False, True, -1, 13753, '727', '1', 1663, '{=Tc/postgres,postgres=CTc/postgres,testuser=CTc/postgres}')


In [4]:
# создаем свою базу данных
# как обычно делаем импорт
import psycopg2 as pg_driver

# подключаемся к БД
db = pg_driver.connect(user="postgres",
                       password="546038",
                       host='localhost',
                       port='5432')


def execute_queries(db, sql_commands):
    db.autocommit = True
    # тот же самый курсор
    with db.cursor() as cursor:
        # в цикле поочередно выполняем запросы
        for sql_command in sql_commands:
            print(sql_command)
            # тот же самый .execute() для выполнения запроса
            cursor.execute(sql_command)

# создаем список запросов к бд
# Здесь у нас идут 3 последовательных команды
# 1. Создать свою базу данных с именем testdba
# 2. Создать пользователя с паролем
# 3. Выдать все доступы к базе данных

# До это момента у нас была база данных по умолчанию, дефолтная, но это плохая практика использовать дефолтную базу, поэтому
# обычно создают отдельного пользователя. Например, у вас отдельная база, пользователь и таблицы для хранения данных по социологическому опросу,
# отдельная база с пользователем для дипломной работы, отдельная база для курсовой работы по нашему предметы. Это удобный способ разграничить
# набор таблиц для определенной сферы
sql_commands = ["CREATE DATABASE testdb;",
                "CREATE USER testuser with encrypted password 'testuser';",
                "GRANT ALL PRIVILEGES ON DATABASE testdb TO testuser;"]


# запускаем выполнение запросов
execute_queries(db, sql_commands)

CREATE DATABASE testdb;


DuplicateDatabase: ОШИБКА:  база данных "testdb" уже существует


In [5]:
import psycopg2 as pg_driver

db = pg_driver.connect(
    database="testdb", 
    user='testuser',
    password='testuser', 
    host='localhost', 
    port='5432'
)


def execute_queries(db, sql_commands):
    db.autocommit = True
    # тот же самый курсор
    with db.cursor() as cursor:
        # в цикле поочередно выполняем запросы
        for sql_command in sql_commands:
            print(sql_command)
            # тот же самый .execute() для выполнения запроса
            cursor.execute(sql_command)


sql_commands = ["DROP TABLE IF EXISTS user_history;",
                "DROP TABLE IF EXISTS user_payment;",
                """CREATE TABLE user_history (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                """,
                """CREATE TABLE user_payment (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         sum_rub    INT       NOT NULL
                  );
                """]


execute_queries(db, sql_commands)

DROP TABLE IF EXISTS user_history;
DROP TABLE IF EXISTS user_payment;
CREATE TABLE user_history (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                
CREATE TABLE user_payment (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         sum_rub    INT       NOT NULL
                  );
                


После выполнения предыдущей ячейки мы ожидаем, что у нас было создано 2 таблицы. Давайте проверим, есть ли эти таблицы путем подсчета количества строк внутри них, потому что если таблиц нет, то запросы упадут с ошибкой

Для начала создадим метод, который позволит нам компактно выполнять запросы, чтобы не приходилось постоянно что-то копировать

In [2]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = psycopg2.connect(
                                database="testdb", 
                                user='testuser',
                                password='testuser', 
                                host='localhost', 
                                port='5432'
                     )
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()

In [7]:
row_count_hist = execute_query("select count(*) from user_history", fetch_result=True)
row_count_payment = execute_query("select count(*) from user_payment", fetch_result=True)

print(row_count_hist)
print(row_count_payment)

[Record(count=0)]
[Record(count=0)]


Видим выше, что никаких ошибок нет, а это значит, что у нас есть пустые таблицы, ими можно пользоватьсы :)

In [10]:
# заносим данные в таблицу
# выполнили этот запрос дважды, поэтому получилась таблица с дубликатами
query = """ INSERT INTO user_payment (user_id, created, sum_rub) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 100000),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 150000),
                 (3, to_timestamp('16-07-2020 09:36:38', 'dd-mm-yyyy hh24:mi:ss'), 200000)
            
        """

execute_query(query)

In [11]:
all_rows = execute_query("select * from user_payment", fetch_result=True)

for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, created=datetime.datetime(2021, 5, 16, 15, 36, 38), sum_rub=100000)
1 Record(user_id=2, created=datetime.datetime(2018, 6, 16, 14, 36, 38), sum_rub=150000)
2 Record(user_id=3, created=datetime.datetime(2020, 7, 16, 9, 36, 38), sum_rub=200000)
3 Record(user_id=1, created=datetime.datetime(2021, 5, 16, 15, 36, 38), sum_rub=100000)
4 Record(user_id=2, created=datetime.datetime(2018, 6, 16, 14, 36, 38), sum_rub=150000)
5 Record(user_id=3, created=datetime.datetime(2020, 7, 16, 9, 36, 38), sum_rub=200000)


In [12]:
# находим пользователей, у которых сумма больше или равна 150000
users_with_more_150000 = execute_query("select user_id from user_payment where sum_rub >= 150000", fetch_result=True)
print(users_with_more_150000)

# смотрим на все суммы по айдишникам
all_rows = execute_query("select user_id, sum_rub from user_payment", fetch_result=True)
print("-----")

# создаем функцию, чтобы результаты предыдущего запроса красиво вывелись
def print_pretty(rows):
    title = 'user_id,sum_rub'
    values = "\n".join(["{},{}".format(elem.user_id, elem.sum_rub) for elem in rows])
    
    print(f"{title}\n{values}")

print_pretty(all_rows)

[Record(user_id=2), Record(user_id=3), Record(user_id=2), Record(user_id=3)]
-----
user_id,sum_rub
1,100000
2,150000
3,200000
1,100000
2,150000
3,200000


In [13]:
# теперь ищем то же самое, но выводим только уникальные значения
all_rows = execute_query("select distinct user_id from user_payment where sum_rub >= 150000", fetch_result=True)
print(all_rows)

all_rows = execute_query("select distinct sum_rub from user_payment", fetch_result=True)
print(all_rows)

[Record(user_id=3), Record(user_id=2)]
[Record(sum_rub=100000), Record(sum_rub=200000), Record(sum_rub=150000)]


In [14]:
# просто прогоняем разные запросы
result = execute_query("select sum(sum_rub) from user_payment", fetch_result=True)
print(result)
result = execute_query("select avg(sum_rub) from user_payment", fetch_result=True)
print(result)
result = execute_query("select max(sum_rub) from user_payment", fetch_result=True)
print(result)
result = execute_query("select min(sum_rub) from user_payment", fetch_result=True)
print(result)

[Record(sum=900000)]
[Record(avg=Decimal('150000.000000000000'))]
[Record(max=200000)]
[Record(min=100000)]


In [15]:
# максимальная дата - самая новая, минимальная - самая старая
result = execute_query("select max(created) from user_payment", fetch_result=True)
print(result)

result = execute_query("select min(created) from user_payment", fetch_result=True)
print(result)

result = execute_query("select created from user_payment", fetch_result=True)
for row in result:
    print(row.created)

[Record(max=datetime.datetime(2021, 5, 16, 15, 36, 38))]
[Record(min=datetime.datetime(2018, 6, 16, 14, 36, 38))]
2021-05-16 15:36:38
2018-06-16 14:36:38
2020-07-16 09:36:38
2021-05-16 15:36:38
2018-06-16 14:36:38
2020-07-16 09:36:38


In [16]:
# упорядочим по убыванию по дате
result = execute_query("select * from user_payment order by created desc", fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

0 Record(user_id=1, created=datetime.datetime(2021, 5, 16, 15, 36, 38), sum_rub=100000)
1 Record(user_id=1, created=datetime.datetime(2021, 5, 16, 15, 36, 38), sum_rub=100000)
2 Record(user_id=3, created=datetime.datetime(2020, 7, 16, 9, 36, 38), sum_rub=200000)
3 Record(user_id=3, created=datetime.datetime(2020, 7, 16, 9, 36, 38), sum_rub=200000)
4 Record(user_id=2, created=datetime.datetime(2018, 6, 16, 14, 36, 38), sum_rub=150000)
5 Record(user_id=2, created=datetime.datetime(2018, 6, 16, 14, 36, 38), sum_rub=150000)


In [17]:
# упорядочим по возрастанию по айдишнику,asc можно не прописывать, стоит по умолчанию
result = execute_query("select distinct user_id from user_payment order by user_id asc", fetch_result=True)
for row, value in enumerate(result):
    print(row, value)

0 Record(user_id=1)
1 Record(user_id=2)
2 Record(user_id=3)


In [18]:
# добавляем новые строки в таблицу
query = """ INSERT INTO user_payment (user_id, created, sum_rub) 
            VALUES 
                 (1, to_timestamp('08-04-2017 11:36:38', 'dd-mm-yyyy hh24:mi:ss'), 100000),
                 (2, to_timestamp('12-01-2008 10:36:38', 'dd-mm-yyyy hh24:mi:ss'), 150000),
                 (3, to_timestamp('11-12-2022 23:36:38', 'dd-mm-yyyy hh24:mi:ss'), 200000)
            
        """

execute_query(query)

In [19]:
# проверяем, появились или нет
all_rows = execute_query("select * from user_payment", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)
    
    
all_rows = execute_query("select distinct user_id from user_payment", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)    

0 Record(user_id=1, created=datetime.datetime(2021, 5, 16, 15, 36, 38), sum_rub=100000)
1 Record(user_id=2, created=datetime.datetime(2018, 6, 16, 14, 36, 38), sum_rub=150000)
2 Record(user_id=3, created=datetime.datetime(2020, 7, 16, 9, 36, 38), sum_rub=200000)
3 Record(user_id=1, created=datetime.datetime(2021, 5, 16, 15, 36, 38), sum_rub=100000)
4 Record(user_id=2, created=datetime.datetime(2018, 6, 16, 14, 36, 38), sum_rub=150000)
5 Record(user_id=3, created=datetime.datetime(2020, 7, 16, 9, 36, 38), sum_rub=200000)
6 Record(user_id=1, created=datetime.datetime(2017, 4, 8, 11, 36, 38), sum_rub=100000)
7 Record(user_id=2, created=datetime.datetime(2008, 1, 12, 10, 36, 38), sum_rub=150000)
8 Record(user_id=3, created=datetime.datetime(2022, 12, 11, 23, 36, 38), sum_rub=200000)
0 Record(user_id=3)
1 Record(user_id=2)
2 Record(user_id=1)


In [20]:
# считаем, сколько у нас одинаковых айдишников по каждой группе, группируем по айдишнику
all_rows = execute_query("select user_id as uid, count(*) as payment_count from user_payment group by user_id", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(uid=3, payment_count=3)
1 Record(uid=2, payment_count=3)
2 Record(uid=1, payment_count=3)


In [21]:
# считаем количество пользователей, которым выплатили по 100000, 150000 и 200000, группируем по сумме
all_rows = execute_query("select sum_rub, count(*) as user_count from user_payment group by sum_rub", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(sum_rub=100000, user_count=3)
1 Record(sum_rub=200000, user_count=3)
2 Record(sum_rub=150000, user_count=3)


In [22]:
# когда каждому пользователю впервые были выплачены деньги
all_rows = execute_query("select user_id, min(created) as first_payment_date from user_payment group by user_id", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value.user_id, value.first_payment_date)

0 3 2020-07-16 09:36:38
1 2 2008-01-12 10:36:38
2 1 2017-04-08 11:36:38


In [23]:
# практика в создании таблиц и наполнение их данными
query = """CREATE TABLE item (
                         id         INT       NOT NULL,
                         name       TEXT      NOT NULL,
                         price      INT       NOT NULL
                         
                  );
        """

execute_query(query)


query = """CREATE TABLE item_delivery (
                         item_id    INT       NOT NULL,
                         dlvr_time  TIMESTAMP NOT NULL,
                         cnt        INT       NOT NULL
                         
                  );
        """

execute_query(query)


query = """ INSERT INTO item (id, name, price) 
            VALUES 
                 (1, 'Juice', 150),
                 (2, 'Apple', 200),
                 (3, 'Orange', 120)
            
        """

execute_query(query)


query = """ INSERT INTO item_delivery (item_id, dlvr_time, cnt) 
            VALUES 
                 (1, to_timestamp('08-04-2017 11:36:38', 'dd-mm-yyyy hh24:mi:ss'), 10),
                 (2, to_timestamp('12-01-2008 10:36:38', 'dd-mm-yyyy hh24:mi:ss'), 20),
                 (3, to_timestamp('11-12-2022 23:36:38', 'dd-mm-yyyy hh24:mi:ss'), 30),
                 (2, to_timestamp('14-01-2008 10:36:38', 'dd-mm-yyyy hh24:mi:ss'), 10),
                 (3, to_timestamp('16-12-2022 23:36:38', 'dd-mm-yyyy hh24:mi:ss'), 15)
            
        """

execute_query(query)


print(execute_query("select * from item", fetch_result=True))
print(execute_query("select * from item_delivery", fetch_result=True))

[Record(id=1, name='Juice', price=150), Record(id=2, name='Apple', price=200), Record(id=3, name='Orange', price=120)]
[Record(item_id=1, dlvr_time=datetime.datetime(2017, 4, 8, 11, 36, 38), cnt=10), Record(item_id=2, dlvr_time=datetime.datetime(2008, 1, 12, 10, 36, 38), cnt=20), Record(item_id=3, dlvr_time=datetime.datetime(2022, 12, 11, 23, 36, 38), cnt=30), Record(item_id=2, dlvr_time=datetime.datetime(2008, 1, 14, 10, 36, 38), cnt=10), Record(item_id=3, dlvr_time=datetime.datetime(2022, 12, 16, 23, 36, 38), cnt=15)]


Есть пользователи, которые ставят друг другу лайки. Создать таблицы для хранения инфы. Написать запрос, который выведет id пользователя, имя, лайков получено, лайков поставлено, взаимных лайков. Написать запрос, который посчитает 5 самых популярных пользователей

In [3]:
import psycopg2 as pg_driver

db = pg_driver.connect(
    database="testdb", 
    user='testuser',
    password='testuser', 
    host='localhost', 
    port='5432'
)


def execute_queries(db, sql_commands):
    db.autocommit = True
    # тот же самый курсор
    with db.cursor() as cursor:
        # в цикле поочередно выполняем запросы
        for sql_command in sql_commands:
            print(sql_command)
            # тот же самый .execute() для выполнения запроса
            cursor.execute(sql_command)
            
# создаем две таблицы пользователи и лайки           
sql_commands = ["DROP TABLE IF EXISTS users;",
                "DROP TABLE IF EXISTS likes;",
                """CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                """,
                """CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                """]


execute_queries(db, sql_commands)

DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS likes;
CREATE TABLE users (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL,
                         created    TIMESTAMP NOT NULL
                );
                
CREATE TABLE likes (
                         user_id    INT       NOT NULL,
                         created    TIMESTAMP NOT NULL,
                         user_id_give    INT       NOT NULL
                  );
                


In [2]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = pg_driver.connect(
                     database="testdb", 
                     user='testuser',
                     password='testuser', 
                     host='localhost', 
                     port='5432'
                 );
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()

# проверяем, что пустые таблицы созданы
row_count_hist = execute_query("select count(*) from users", fetch_result=True)
row_count_payment = execute_query("select count(*) from likes", fetch_result=True)

print(row_count_hist)
print(row_count_payment)

[Record(count=0)]
[Record(count=0)]


In [4]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Mark'),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Dima'),
                 (3, to_timestamp('16-07-2020 09:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Phill'),
                 (4, to_timestamp('16-06-2018 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Viktor'),
                 (5, to_timestamp('16-06-2018 16:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Ivan')
                 
            
        """

execute_query(query)

query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4),
                 (3, to_timestamp('16-07-2020 09:36:38', 'dd-mm-yyyy hh24:mi:ss'), 3),
                 (4, to_timestamp('16-06-2018 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 2),
                 (5, to_timestamp('16-06-2018 16:36:38', 'dd-mm-yyyy hh24:mi:ss'), 1)
                 
            
        """

execute_query(query)

In [5]:
query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (1, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4),
                 (1, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5),
                 (2, to_timestamp('16-06-2018 14:36:38', 'dd-mm-yyyy hh24:mi:ss'), 5)
                 

                 
            
        """

execute_query(query)

In [6]:
# считаем, сколько раз один и тот же пользователь встречается в таблице лайки, групируем по пользователям
all_rows = execute_query("select user_id, count(*) from likes group by user_id", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=3, count=1)
1 Record(user_id=5, count=1)
2 Record(user_id=4, count=1)
3 Record(user_id=2, count=2)
4 Record(user_id=1, count=3)


In [7]:
# достаем эти данные только для первого пользователя
all_rows = execute_query("select user_id, count(*) from likes where user_id=1 group by user_id", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, count=3)


In [8]:
# соединяем таблицы. из левой (упоминается первой - лайки) попадут все данные. если во второй нет соответствия первой, 
# в этих строках будет null
query = """select l.user_id, u.name 
           from likes l
           left join users u
           on l.user_id=u.user_id
           where l.user_id=1 
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, name='Mark')
1 Record(user_id=1, name='Mark')
2 Record(user_id=1, name='Mark')


In [8]:
# узнаем, кто кому ставил лайки. джойним два раза таблицу пользователи, но приравниваем к имени пользователя разные стобцы
# из таблицы лайки. чтобы не запутаться, присваиваем в начале разные имена одной и той же колонке имя пользователя u.name и us.name
query = """select
                l.user_id as from_id,
                u.name as from_name,
                l.user_id_give as to_id,
                us.name as to_name
           from likes l
           left join users u
           on l.user_id=u.user_id
           left join users us
           on l.user_id_give=us.user_id
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(from_id=5, from_name='Ivan', to_id=1, to_name='Mark')
1 Record(from_id=4, from_name='Viktor', to_id=2, to_name='Dima')
2 Record(from_id=3, from_name='Phill', to_id=3, to_name='Phill')
3 Record(from_id=2, from_name='Dima', to_id=4, to_name='Viktor')
4 Record(from_id=1, from_name='Mark', to_id=4, to_name='Viktor')
5 Record(from_id=1, from_name='Mark', to_id=5, to_name='Ivan')
6 Record(from_id=2, from_name='Dima', to_id=5, to_name='Ivan')
7 Record(from_id=1, from_name='Mark', to_id=5, to_name='Ivan')


In [9]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (10, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Mark')
 
        """

execute_query(query)

query = """ INSERT INTO likes (user_id, created, user_id_give) 
            VALUES 
                 (10, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 4)
            
        """

execute_query(query)

In [10]:
# считаем, сколько каждый пользователь поставил лайков, группируем по айди пользователя, иначе ругается
query = """select
                l.user_id as from_id,
                u.name as from_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id=u.user_id
           group by l.user_id, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(from_id=4, from_name='Viktor', like_count=1)
1 Record(from_id=1, from_name='Mark', like_count=3)
2 Record(from_id=3, from_name='Phill', like_count=1)
3 Record(from_id=10, from_name='Mark', like_count=1)
4 Record(from_id=2, from_name='Dima', like_count=2)
5 Record(from_id=5, from_name='Ivan', like_count=1)


In [11]:
# но так у нас Марк получился два раза. чтоб этого не было, группируем по имени пользователя и чтоб не ругалось, 
# закоментируем графу айди с помощью --
query = """select
                --l.user_id as from_id,
                u.name as from_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id=u.user_id
           group by u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(from_name='Viktor', like_count=1)
1 Record(from_name='Phill', like_count=1)
2 Record(from_name='Ivan', like_count=1)
3 Record(from_name='Mark', like_count=4)
4 Record(from_name='Dima', like_count=2)


In [12]:
# считаем, сколько каждый пользователь получил лайков
query = """select
                l.user_id_give as to_id,
                u.name as to_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id_give=u.user_id
           group by l.user_id_give, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(to_id=4, to_name='Viktor', like_count=3)
1 Record(to_id=1, to_name='Mark', like_count=1)
2 Record(to_id=3, to_name='Phill', like_count=1)
3 Record(to_id=2, to_name='Dima', like_count=1)
4 Record(to_id=5, to_name='Ivan', like_count=3)


In [13]:
query = """select *
                
           from likes l
           order by l.user_id_give
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=5, created=datetime.datetime(2018, 6, 16, 16, 36, 38), user_id_give=1)
1 Record(user_id=4, created=datetime.datetime(2018, 6, 16, 15, 36, 38), user_id_give=2)
2 Record(user_id=3, created=datetime.datetime(2020, 7, 16, 9, 36, 38), user_id_give=3)
3 Record(user_id=10, created=datetime.datetime(2021, 5, 16, 15, 36, 38), user_id_give=4)
4 Record(user_id=2, created=datetime.datetime(2018, 6, 16, 14, 36, 38), user_id_give=4)
5 Record(user_id=1, created=datetime.datetime(2021, 5, 16, 15, 36, 38), user_id_give=4)
6 Record(user_id=1, created=datetime.datetime(2018, 6, 16, 14, 36, 38), user_id_give=5)
7 Record(user_id=2, created=datetime.datetime(2018, 6, 16, 14, 36, 38), user_id_give=5)
8 Record(user_id=1, created=datetime.datetime(2021, 5, 16, 15, 36, 38), user_id_give=5)


In [14]:
query = """ INSERT INTO users (user_id, created, name) 
            VALUES 
                 (8, to_timestamp('16-05-2021 15:36:38', 'dd-mm-yyyy hh24:mi:ss'), 'Damir')
 
        """

execute_query(query)

In [15]:
# находим пользователя, который не поставил ни одного лайка
query = """select
                
                u.user_id as user_id,
                u.name as user_name,
                l.user_id_give as to_id
                
                
           from users u
           left join likes l
           on u.user_id=l.user_id
           where l.user_id_give is null
           order by u.user_id  
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=8, user_name='Damir', to_id=None)


In [28]:
# считаем взаимные лайки
query = """select t1.user_id, t1.user_id_give,
           count(*) as like_count
                
           from likes t1
           inner join likes t2
           on t1.user_id_give=t2.user_id
           where t2.user_id_give=t1.user_id 
           group by t1.user_id_give, t1.user_id
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, user_id_give=5, like_count=2)
1 Record(user_id=2, user_id_give=4, like_count=1)
2 Record(user_id=3, user_id_give=3, like_count=1)
3 Record(user_id=4, user_id_give=2, like_count=1)
4 Record(user_id=5, user_id_give=1, like_count=2)


In [22]:
# выводим 5 самых популярных пользователей
query = """select
                l.user_id_give as to_id,
                u.name as to_name,
                count(*) as like_count
                
           from likes l
           left join users u
           on l.user_id_give=u.user_id
           group by l.user_id_give, u.name
           order by l.user_id_give DESC
           limit 5
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)   

0 Record(to_id=5, to_name='Ivan', like_count=3)
1 Record(to_id=4, to_name='Viktor', like_count=3)
2 Record(to_id=3, to_name='Phill', like_count=1)
3 Record(to_id=2, to_name='Dima', like_count=1)
4 Record(to_id=1, to_name='Mark', like_count=1)


В воображаемой социальной сети есть Пользователи (айди, имя), Фото (айди, название, автор) и Комментарии к фото (айди, текст, автор, айди фото). Необходимо добавить возможность для пользователей ставить лайки другим пользователям, фото или комментариям к фото. Нужно реализовать такие возможности:
пользователь имеет право отозвать лайк;
необходимо иметь возможность посчитать число полученных сущностью лайков и вывести список пользователей, поставивших лайки;
в будущем могут появиться новые виды сущностей, которые можно лайкать.

In [38]:
import psycopg2 as pg_driver

db = pg_driver.connect(
    database="testdb", 
    user='testuser',
    password='testuser', 
    host='localhost', 
    port='5432'
)


def execute_queries(db, sql_commands):
    db.autocommit = True
    # тот же самый курсор
    with db.cursor() as cursor:
        # в цикле поочередно выполняем запросы
        for sql_command in sql_commands:
            print(sql_command)
            # тот же самый .execute() для выполнения запроса
            cursor.execute(sql_command)
            
sql_commands = ["DROP TABLE IF EXISTS users2;",
                "DROP TABLE IF EXISTS photos;",
                "DROP TABLE IF EXISTS comments;"
                "DROP TABLE IF EXISTS likes;",
                """CREATE TABLE users2 (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL
                );
                """,
                """CREATE TABLE photos (
                         photo_id    INT       NOT NULL,
                         photo_name    TEXT       NOT NULL,
                         author_id    INT       NOT NULL
                  );
                """,
               """CREATE TABLE comments (
                         comment_id    INT       NOT NULL,
                         user_id    INT       NOT NULL,
                         text    TEXT       NOT NULL,
                         author_id    INT       NOT NULL,
                         photo_id    INT       NOT NULL
                );
                """,
               """CREATE TABLE likes (
                         user_from    INT       NOT NULL,
                         entity_type       TEXT       NOT NULL,
                         entity_id    INT       NOT NULL,
                         user_to     INT       NOT NULL    
                         
                );
                """]


execute_queries(db, sql_commands)

DROP TABLE IF EXISTS users2;
DROP TABLE IF EXISTS photos;
DROP TABLE IF EXISTS comments;DROP TABLE IF EXISTS likes;
CREATE TABLE users2 (
                         user_id    INT       NOT NULL,
                         name    TEXT       NOT NULL
                );
                
CREATE TABLE photos (
                         photo_id    INT       NOT NULL,
                         photo_name    TEXT       NOT NULL,
                         author_id    INT       NOT NULL
                  );
                
CREATE TABLE comments (
                         comment_id    INT       NOT NULL,
                         user_id    INT       NOT NULL,
                         text    TEXT       NOT NULL,
                         author_id    INT       NOT NULL,
                         photo_id    INT       NOT NULL
                );
                
CREATE TABLE likes (
                         user_from    INT       NOT NULL,
                         entity_type       TEXT       NOT NUL

In [39]:
import psycopg2
from psycopg2 import Error
from psycopg2.extras import NamedTupleCursor

def execute_query(query, fetch_result=False):
    try:
        connection = pg_driver.connect(
                     database="testdb", 
                     user='testuser',
                     password='testuser', 
                     host='localhost', 
                     port='5432'
                 );
        
        connection.autocommit = True
        cursor = connection.cursor(cursor_factory=NamedTupleCursor)
        cursor.execute(query)
        if fetch_result:
            return cursor.fetchall()
    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)
    finally:
        if (connection):
            cursor.close()
            connection.close()

# проверяем, что пустые таблицы созданы
row_count_hist = execute_query("select count(*) from users2", fetch_result=True)
row_count_payment = execute_query("select count(*) from photos", fetch_result=True)
row_count_comm = execute_query("select count(*) from comments", fetch_result=True)
row_count_lik = execute_query("select count(*) from likes", fetch_result=True)

print(row_count_hist)
print(row_count_payment)
print(row_count_comm)
print(row_count_lik)

[Record(count=0)]
[Record(count=0)]
[Record(count=0)]
[Record(count=0)]


In [40]:
query = """ INSERT INTO users2 (user_id, name) 
            VALUES 
                 (1, 'Mark'),
                 (2, 'Dima'),
                 (3, 'Pina'),
                 (4, 'Alice'),
                 (5, 'Rob'),
                 (6, 'Ann'),
                 (7, 'Jin')
                 
            
        """

execute_query(query)

query = """ INSERT INTO photos (photo_id, photo_name, author_id ) 
            VALUES 
                 (1, 'Forest', 5),
                 (2, 'River', 4),
                 (3, 'City', 3),
                 (4, 'Portret', 7),
                 (5, 'Street', 1),
                 (6, 'Member', 7),
                 (7, 'Home', 4),
                 (8, 'Peach', 6),
                 (9, 'Flowers', 4),
                 (10, 'Cats', 2),
                 (11, 'Dogs', 2)
                 
            
        """

execute_query(query)

query = """ INSERT INTO comments (comment_id, user_id, text, author_id, photo_id ) 
            VALUES 
                 (1, 1, 'Cool', 6, 8),
                 (2, 7, 'Cute', 2, 10),
                 (3, 4, 'Want to be there', 5, 1),
                 (4, 4, 'WWH', 7, 4),
                 (5, 5, 'Beautiful', 4, 9)
                 
            
        """

execute_query(query)

query = """ INSERT INTO likes (user_from, entity_type, entity_id, user_to) 
            VALUES 
                 (1, 'person', 3, 3),
                 (2, 'photo', 5, 1),
                 (3, 'comment', 3, 4),
                 (4, 'photo', 8, 6),
                 (5, 'person', 7, 7)
            
        """

execute_query(query)

In [41]:
# проверяем, появились или нет
all_rows = execute_query("select * from users2", fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_id=1, name='Mark')
1 Record(user_id=2, name='Dima')
2 Record(user_id=3, name='Pina')
3 Record(user_id=4, name='Alice')
4 Record(user_id=5, name='Rob')
5 Record(user_id=6, name='Ann')
6 Record(user_id=7, name='Jin')


In [47]:
# посчитаем, сколько фотографий сделал каждый пользователь
query = """select
                u.name,
                --p.photo_name,
                count(*) as photo_count
                
           from users2 u
           left join photos p
           on u.user_id=p.author_id
           group by u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(name='Pina', photo_count=1)
1 Record(name='Mark', photo_count=1)
2 Record(name='Ann', photo_count=1)
3 Record(name='Alice', photo_count=3)
4 Record(name='Jin', photo_count=2)
5 Record(name='Rob', photo_count=1)
6 Record(name='Dima', photo_count=2)


In [48]:
# посчитаем, сколько лайков поставил каждый пользователь
query = """select
                l.user_from,
                u.name,
                count(*) as like_count
                
           from likes l
           left join users2 u
           on l.user_from=u.user_id
           group by l.user_from, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_from=1, name='Mark', like_count=1)
1 Record(user_from=3, name='Pina', like_count=1)
2 Record(user_from=2, name='Dima', like_count=1)
3 Record(user_from=5, name='Rob', like_count=1)
4 Record(user_from=4, name='Alice', like_count=1)


In [49]:
# посчитаем, сколько лайков получил каждый пользователь (независимо от того за персону, фото или комментарий)
query = """select
                l.user_to,
                u.name,
                count(*) as like_count
                
           from likes l
           left join users2 u
           on l.user_to=u.user_id
           group by l.user_to, u.name
           
        """
all_rows = execute_query(query, fetch_result=True)
for row, value in enumerate(all_rows):
    print(row, value)

0 Record(user_to=1, name='Mark', like_count=1)
1 Record(user_to=7, name='Jin', like_count=1)
2 Record(user_to=3, name='Pina', like_count=1)
3 Record(user_to=4, name='Alice', like_count=1)
4 Record(user_to=6, name='Ann', like_count=1)
